### v2 - RAG - Dorna-Llama3-8B-Instruct-Quantized

In [2]:
import sys
import time
import random
import chromadb
from llama_cpp import Llama
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

/home/amyrmahdy/GitHub/chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
context1 = '''
شرکت نواکلود سه سرویس اصلی ارائه می‌دهد:  
1. نوااستورج (NovaStorage) – یک سرویس ذخیره‌سازی ابری که برای شرکت‌های بزرگ طراحی شده است و امکان رمزگذاری سرتاسری (E2EE) و پشتیبان‌گیری خودکار را دارد.  
2. نواکامپیوتر (NovaCompute) – یک سرویس پردازش ابری که از پردازنده‌های ZetaCore X200 استفاده می‌کند و برای مدل‌های یادگیری ماشین سنگین بهینه‌سازی شده است.  
3. نواکانکت (NovaConnect) – یک پلتفرم شبکه خصوصی ابری (VPC) که به شرکت‌ها امکان ایجاد زیرشبکه‌های ایزوله با IP ثابت خصوصی را می‌دهد.  
'''


context2 = '''
نواکلود سه طرح قیمت‌گذاری ارائه می‌دهد:  
- طرح پایه (Basic): شامل ۵۰ گیگابایت فضای ذخیره‌سازی و ۲ هسته پردازشی با هزینه‌ی ۱۵ دلار در ماه  
- طرح حرفه‌ای (Pro): شامل ۵۰۰ گیگابایت فضای ذخیره‌سازی، ۸ هسته پردازشی و ترافیک نامحدود با هزینه‌ی ۶۰ دلار در ماه  
- طرح سازمانی (Enterprise): شامل ۵ ترابایت فضای ذخیره‌سازی، ۳۲ هسته پردازشی، و قابلیت تنظیم شبکه خصوصی اختصاصی با هزینه‌ی ۲۰۰ دلار در ماه  
'''


context3 = '''
نواکلود امنیت داده‌ها را با سه مکانیزم کلیدی تضمین می‌کند:  
- رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده در NovaStorage  
- احراز هویت چندمرحله‌ای (MFA) برای ورود به تمامی سرویس‌ها  
- فایروال هوشمند که تنها IPهای تأیید‌شده را به شبکه NovaConnect متصل می‌کند  
'''



context4  ='''
در سه ماه گذشته، NovaCompute در ۹۸.۹٪ مواقع بدون اختلال کار کرده است، اما یک قطعی ۲ ساعته در تاریخ ۱۵ فوریه ۲۰۲۴ به دلیل بروزرسانی سخت‌افزاری رخ داده است. در همین مدت، NovaStorage بدون هیچ اختلالی فعال بوده است.  
'''

In [4]:
chunks = [context1,context2,context3,context4]

In [5]:
# Load model from path
# 'all-MiniLM-L6-v2'
embedding_model_name = 'all-MiniLM-L6-v2'
embedding_model_path = f"./models/{embedding_model_name}"
embedding_model = SentenceTransformer(embedding_model_path)
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=embedding_model_name
)


In [10]:
# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chromadb")

# Delete the collection if needed
# client.delete_collection("endpoint_debugging")
# Create a collection for debugging
collection = client.get_or_create_collection(
                    name = "endpoint_debugging",
                    metadata={"hnsw:space": "cosine"},
                    embedding_function=sentence_transformer_ef
            )

In [11]:
embeddings = embedding_model.encode(chunks)

collection.add(
    embeddings=embeddings,
    documents=chunks,
    ids= [str(i) for i in range(len(chunks))]
)

In [12]:
# Query ChromaDB
number_of_results = 1

def query_chromadb(user_query):
    query_embedding = embedding_model.encode([user_query])
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=number_of_results
    )
    print(f'distances: {results["distances"][0]}')
    return results["documents"][0] if results["documents"] else ["No relevant information found."]  # Top 3 relevant sentences


In [13]:
# questions1 = '''
# کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟  
# پردازنده‌ی استفاده‌شده در NovaCompute چیست؟  
# آیا NovaConnect امکان داشتن IP ثابت را فراهم می‌کند؟  
# '''

# questions2 = '''
# در طرح Pro چند هسته پردازشی ارائه می‌شود؟  
# کدام طرح قابلیت تنظیم شبکه خصوصی دارد؟  
# هزینه‌ی ماهانه‌ی طرح پایه چقدر است؟  
# '''

# questions3 = '''
# کدام سرویس نواکلود از رمزگذاری سرتاسری پشتیبانی می‌کند؟  
# آیا احراز هویت چندمرحله‌ای (MFA) در نواکلود اجباری است؟  
# چگونه NovaConnect امنیت دسترسی به شبکه را تضمین می‌کند؟  
# '''

# questions4 = '''
# چند درصد از زمان، NovaCompute در سه ماه گذشته فعال بوده است؟  
# آخرین قطعی NovaCompute در چه تاریخی رخ داده و چه مدت طول کشیده است؟  
# کدام سرویس در سه ماه گذشته هیچ قطعی نداشته است؟  
# '''

In [14]:
question_example1 = 'کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟.'
query_chromadb(question_example1)

distances: [0.4030753960556114]


['\nنواکلود امنیت داده\u200cها را با سه مکانیزم کلیدی تضمین می\u200cکند:  \n- رمزگذاری سرتاسری (E2EE) برای داده\u200cهای ذخیره\u200cشده در NovaStorage  \n- احراز هویت چندمرحله\u200cای (MFA) برای ورود به تمامی سرویس\u200cها  \n- فایروال هوشمند که تنها IPهای تأیید\u200cشده را به شبکه NovaConnect متصل می\u200cکند  \n']

In [15]:
question_example2 = 'در طرح Pro چند هسته پردازشی ارائه می‌شود؟'
query_chromadb(question_example2)
# TODO: Test other embedding models

distances: [0.5577748587431057]


['\nدر سه ماه گذشته، NovaCompute در ۹۸.۹٪ مواقع بدون اختلال کار کرده است، اما یک قطعی ۲ ساعته در تاریخ ۱۵ فوریه ۲۰۲۴ به دلیل بروزرسانی سخت\u200cافزاری رخ داده است. در همین مدت، NovaStorage بدون هیچ اختلالی فعال بوده است.  \n']

In [16]:
question_example1_follow_up = 'میشه بیشتر راجع به این سرویس توضیح بدی؟'

In [17]:
model_path = './models/Dorna-Llama3-8B-Instruct-GGUF-Q8/dorna-llama3-8b-instruct.Q8_0.gguf'

llm = Llama(
      model_path=model_path,
      chat_format="llama-3",
      n_gpu_layers=-1,
      n_ctx=2048,
      verbose = False
)


In [18]:
# TODO: consider cosine similarity to be returned and use threshold to avoid retrieving irrelevent documents

In [19]:
TEMPLATE = '''
تاریخچه مکالمات:
{history}

دانش پایه:
{context}

سوال کاربر:
{prompt}
'''

In [20]:
def retriever(PROMPT):
    retrieved_docs = query_chromadb(PROMPT)
    CONTEXT = "\n".join(retrieved_docs)
    # print(f"Retrieved docs: {CONTEXT}","-----")
    return CONTEXT

In [21]:
def generator(model_input): # , endpoint_name
    
    messages = [
    {"role": "system", "content": "تو یک دستیار متخصص  و پشتیبانی فنی وضعیت سرویس ها هستی که با توجه به دانش پایه، به کاربر پاسخ کمک کننده میدی"},
    {"role": "user", "content": f"{model_input}"},
    ]   
    
    full_result = llm.create_chat_completion(
    messages = messages,
    top_p=0.85,
    temperature=0.1
    )
    response = full_result['choices'][0]['message']['content']
    
    return response


In [22]:
conversation_history = []
def chat(PROMPT):
    CONTEXT = retriever(PROMPT)

    HISTORY = "\n".join(conversation_history)

    model_input = TEMPLATE.format(history=HISTORY,context=CONTEXT, prompt=PROMPT)

    response = generator(model_input)

    conversation_history.append(f"سوال کاربر: {PROMPT}")
    conversation_history.append(f"پاسخ کمک کننده: {response}")    

    return response


In [40]:
# TODO: dynamic delay should be fine-tuned (random.uniform(0.02, 0.08))

In [35]:
def stream_print(text):
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        
        # Dynamic delay
        # time.sleep(random.uniform(0.02, 0.08))  # Adjust range for a natural effect
        time.sleep(random.uniform(0.02, 0.08))
    print()  # Move to the next line after completion

In [ ]:
# Stream
print(question_example1)
start = time.time()
conversation_history = []
response = chat(question_example1)
stream_print(response)
end = time.time()
print('---\nall done at',end - start,'seconds')

کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟.
distances: [0.4030753960556114]
بر اساس دانش پایه، سرویس NovaStorage برای ذخیره‌سازی ابری استفاده می‌شود. این سرویس با استفاده از رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده تضمین می‌کند که امنیت داده‌ها حفظ شود.
---
all done at 11.09956407546997 seconds


In [38]:
# Non-Stream
print(question_example1)
start = time.time()
conversation_history = []
response = chat(question_example1)
print(response)
end = time.time()
print('---\nall done at',end - start,'seconds')

کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟.
distances: [0.4030753960556114]
بر اساس دانش پایه، سرویس NovaStorage برای ذخیره‌سازی ابری استفاده می‌شود. این سرویس با استفاده از رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده تضمین می‌کند که امنیت داده‌ها حفظ شود.
---
all done at 1.640636682510376 seconds


In [39]:
print(question_example1_follow_up)
# conversation_history = []
response = chat(question_example1_follow_up)
stream_print(response)

میشه بیشتر راجع به این سرویس توضیح بدی؟
distances: [0.39181215855848395]
با خوشحالی! به عنوان یک دستیار متخصص و پشتیبانی فنی، خوشحالم که می‌توانم به شما در مورد سرویس NovaStorage و امنیت داده‌ها در نواکلود توضیح بدهم.

نواستورج (NovaStorage) یک سرویس ذخیره‌سازی ابری است که توسط نواکلود ارائه می‌شود. این سرویس با استفاده از رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده تضمین می‌کند که امنیت داده‌ها حفظ شود. این بدان معناست که حتی در صورت دسترسی به داده‌ها توسط افراد ناامن، داده‌ها به دلیل رمزگذاری، قابل خواندن و استفاده نخواهند بود.

یکی از مزایای استفاده از NovaStorage، امنیت داده‌ها در طول انتقال و ذخیره است. این بدان معناست که حتی در صورت دسترسی به داده‌ها توسط افراد ناامن در طول انتقال یا ذخیره، داده‌ها به دلیل رمزگذاری، قابل خواندن و استفاده نخواهند بود.

علاوه بر رمزگذاری سرتاسری، نواستورج همچنین از فایروال هوشمند استفاده می‌کند که تنها IPهای تأیید‌شده را به شبکه NovaConnect متصل می‌کند. این بدان معناست که تنها افرادی که دارای دسترسی مجاز هستند، می‌توانند به شبکه متصل شوند و به 

In [41]:
print(question_example2)
conversation_history = []
response = chat(question_example2)
stream_print(response)

در طرح Pro چند هسته پردازشی ارائه می‌شود؟
distances: [0.5577748587431057]
با توجه به اطلاعات ارائه شده، می‌توان به کاربر پاسخ داد:

"در طرح Pro، NovaCompute 8 هسته پردازشی ارائه می‌کند. این تعداد هسته پردازشی به کاربران اجازه می‌دهد تا به طور موثرتر از منابع محاسباتی استفاده کنند و کارهای پیچیده را با سرعت و کارایی بیشتری انجام دهند."
